# 載入所需套件

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm

# 定義爬取104人力銀行職缺資訊函數

In [2]:
def job_crawler(url):
    response=requests.get(url)
    soup=BeautifulSoup(response.text,'html.parser')

    articles=soup.select('article.b-block--top-bord.job-list-item.b-clearfix.js-job-item')
    titles=[e.select('a')[0].text for e in articles]
    companys=[e.select('a')[1].text.replace('\n','').replace(' ','') for e in articles]
    links=['https:{}'.format(e.select('a')[1]['href']) for e in articles]
    statuses=[e.select('a')[2].text.split('應徵')[0] for e in articles]
    dates=[e.select('span')[0].text.replace('\n','').replace(' ','') for e in articles]
    salaries=[e.select('span')[1].text for e in articles]
    locations=[e.select('ul.b-list-inline.b-clearfix.job-list-intro.b-content>li')[0].text for e in articles]
    requirements=[e.select('ul.b-list-inline.b-clearfix.job-list-intro.b-content>li')[1].text for e in articles]
    degrees=[e.select('ul.b-list-inline.b-clearfix.job-list-intro.b-content>li')[2].text for e in articles]
    texts=[e.select('p')[0].text.replace('\r','').replace('\n','').replace('\t','') for e in articles]
    
    dates,industrys=[],[]
    for e in articles:
        if e.select('span')[0].text.replace('\n','').replace(' ','')=='':
            dates.append(np.nan)
        else:
            dates.append(e.select('span')[0].text.replace('\n','').replace(' ',''))
    
    for e in articles:
        if len(e.select_one('ul.b-list-inline.b-clearfix').select('li'))==3:
            industrys.append(e.select_one('ul.b-list-inline.b-clearfix').select('li')[-1].text)
        else:
            industrys.append(np.nan)
    
    df=pd.DataFrame({
        '日期':dates,
        '應徵':statuses,
        '工作':titles,
        '公司':companys,
        '產業':industrys,
        '薪水':salaries,
        '地區':locations,
        '要求':requirements,
        '學歷':degrees,
        '介紹':texts,
        '網址':links
    })
    
    return df

# 定義爬取多頁104人力銀行關鍵字職缺資訊函數

In [3]:
def page_function(keyword,page):
    dfs=[]
    
    for i in tqdm(range(page)):
        url='https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword={}&order=1&asc=0&page={}&mode=s'.format(keyword,i+1)
        d=job_crawler(url)
        dfs.append(d)
    
    df=pd.concat(dfs,ignore_index=True)
    
    return df

# 指定關鍵字和總頁數表格呈現

In [4]:
df=page_function('金融實習',3)
df.head()

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.19s/it]


,日期,應徵,工作,公司,產業,薪水,地區,要求,學歷,介紹,網址
0,2/08,11~30人,【MacroMicro/財經Ｍ平方】金融研究助理 (實習生),恩平方財經股份有限公司,其他投資理財相關業,時薪160~300元,台北市信義區,經歷不拘,專科,主要為協助研究員，包含：- 管理及建立後端各式經濟數據- 搜集市場行情、金融資訊- 製作課程...,https://www.104.com.tw/company/1a2x6bkrzz?jobs...
1,2/14,30人以上,【工讀生】金融數據分析實習 (稽核室),國泰世華商業銀行股份有限公司_人力資源部,銀行業,"月薪23,800~26,000元",台北市信義區,經歷不拘,大學,工作說明：(1)金融數據收集、整理、分析(2)資料視覺化與報表設計其他說明： (1)熟悉EX...,https://www.104.com.tw/company/1xzkino?jobsour...
2,2/10,11~30人,Diploma Legal Practice金融科技實習生（法務）,普匯金融科技股份有限公司,電腦軟體服務業,"時薪158~158,158元",台北市中山區,經歷不拘,大學,1.熟悉金融相關法律2.有專案經驗3.能草擬、修訂基本合約4.專案研究分析5.文章撰寫6.主管交辦,https://www.104.com.tw/company/1a2x6bkhzg?jobs...
3,2/06,0~5人,【實習】財務部 - 學生實習,富強鑫精密工業股份有限公司,精密儀器相關製造業,時薪158元,台南市關廟區,經歷不拘,專科,1.資料統整2.財務事務協助3.會計事務協助4.主管指派工作可全年度實習、一周上班天數3-5...,https://www.104.com.tw/company/13kawv1c?jobsou...
4,2/13,11~30人,金融行銷業務特助-(實習生),永誠國際證券投資顧問股份有限公司,證券及期貨業,"年薪300,000~3,000,000元",台北市中山區,經歷不拘,大學,只要您對投資理財有興趣有膽量，喜歡與人溝通有企圖心，超級想賺錢有夢想，想成為超級業務我們就全...,https://www.104.com.tw/company/1a2x6biulk?jobs...
